<a href="https://colab.research.google.com/github/bhomaidan1990/Computer_Vision_Labs/blob/main/CV_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision Course

## Lab (1)


### Read and parse the original image

In [ ]:
#########################
# @Author: Belal Hmedan #
#########################

#========================
# Import Libraries
#========================
import os
# import glob
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import cv2

#------------------------
# Read Image
#========================
datasetPath = "/media/desertghost/WD/Computer_vision/Lecture_1/Lab_1/Dataset/Datast/"
specialPath = "2002/07/21/big/img_390.jpg"
imPath = os.path.join(datasetPath, specialPath) 
img = cv2.imread(imPath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# print(img.shape)

#------------------------
# show Image
#========================
plt.figure(num='FDDB')

# plt.subplot(221)
plt.imshow(img)
plt.title('Original')
plt.axis('off')

plt.show()

### Read the Annotaion

In [ ]:
#------------------------
# define Path
#========================
foldsPath = "/media/desertghost/WD/Computer_vision/Lecture_1/Lab_1/Dataset/FDDB-folds/"
foldsList  = (os.listdir(foldsPath))

#------------------------
# devide files
#========================
folds = foldsList[1::2] # Evens image names
annot = foldsList[0::2] # Odds annotaions

#------------------------
# Find Corresponding Annotation Fold
#========================
# get the name without the extension
imName = os.path.splitext(specialPath)[0]
# this to find only the string which ends with ths pattern
imName = imName + '$'
# Fold Search Loop
for idx, fold in enumerate(folds):
    foldPath = os.path.join(foldsPath, fold)
    txtFile = open(foldPath, "r")
    for lin in txtFile:
        if(re.findall(imName,lin) != []):
            print('Fold: ',fold,'\nImage Name: ', imName[:-1], lin)
            # Annotaion Fold
            annot_path = os.path.join(foldsPath,annot[idx])
            # print(annot_path)
    # Close the txt file
    txtFile.close()
    
# Initialization
Annotaions = []
AnnotData = []
linIndex = -2
numFaces = 0

# Annot Search Loop
AnnotFile = open(annot_path, "r")
for idx, lin in enumerate(AnnotFile):
    if(re.findall(imName,lin) != []):
        linIndex = idx
    if (idx == (linIndex + 1) ):
        numFaces = int(lin)
        print("number of faces is: ", numFaces)
    if linIndex != -2:
        if idx in range(linIndex + 2, linIndex + 2 + numFaces):
            Annotaions.append(lin)
# print(Annotaions)
for Annotaion in Annotaions:
    AnnotData.append(list(filter(None, list(re.split("\s", Annotaion)))))
AnnotData = np.array(AnnotData, dtype=float )
AnnotData = np.floor(AnnotData[:,:-1]).astype(int)
print(' First Face: \n   max_radius | min_radius | angle | center_x | center_y \n -------------------------------------------------------\n', AnnotData[0])

### Parse the image with the annotaion

In [ ]:
#------------------------
# Elliptic Annotation Drawing Function
#========================
def Elliptic(X, Y, xc, yc, Rmax, Rmin, theta):
    '''
    Function to Draw one Red Ellipses on Black Background
    Parameters:
    @Param: X : orginal image height.
    @Param: Y : orginal image width.
    @Param: xc, yc : Center coordinates of the Ellipse.
    @Param: Rmax, Rmin : max, and min Radius of the Ellipse.
    @Param: theta : Ellipse Rotation Angle 
    '''
    center_coordinates = (xc, yc) 
    
    axesLength = (Rmin, Rmax) 
    
    angle = theta
    
    startAngle = 0
    
    endAngle = 360
    
    # Red color in  RGB
    color = (255, 0, 0) 
    
    # Line thickness of 3 px 
    thickness = 3
    
    # Draw Black image
    image = np.zeros((X, Y, 3))
    # Using cv2.ellipse() method 
    # Draw a ellipse with red line borders of thickness of 5 px 
    result = cv2.ellipse(image, center_coordinates, axesLength, 
            angle, startAngle, endAngle, color, thickness) 
    
    return result 
#------------------------------------------------------------------
#------------------------
# Draw Anottations
#========================
Ellip = np.zeros_like(img)

for row in range(AnnotData.shape[0]):
    Ellip = np.maximum(Ellip, Elliptic(img.shape[0], img.shape[1], AnnotData[row, 3].astype(int),
                         AnnotData[row, 4].astype(int), AnnotData[row, 0].astype(int), AnnotData[row, 1].astype(int), AnnotData[row, 2].astype(int)))
print(img.max(), Ellip.max())
res = np.maximum(img, Ellip.astype(np.uint))
#------------------------
# show Result
#========================
plt.figure(num='FDDB2')

plt.subplot(121)
plt.imshow(img)
plt.title('Original')
plt.axis('off')

plt.subplot(122)
plt.imshow(res)
plt.title('Annotated')
plt.axis('off')

plt.show()